In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql

## Constants

In [2]:
DB_NAME = 'consumer_complaints'
DB_USER = 'db_user'
DB_PASSWORD = 'password'
DB_HOST = 'localhost'
TABLE_NAME = 'complaints'

## Connection to postgres

In [3]:
conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST)
cursor = conn.cursor()
conn.autocommit = True

## Postgres helper functions

In [4]:
def create_table(cursor):
    sql_query = sql.SQL(f'''CREATE TABLE IF NOT EXISTS {TABLE_NAME}(
                        date_received DATE,
                        product_name TEXT,
                        sub_product TEXT,
                        issue TEXT,
                        sub_issue TEXT,
                        consumer_complaint_narrative TEXT,
                        company_public_response TEXT,
                        company TEXT,
                        state_name TEXT,
                        zip_code TEXT,
                        tags TEXT,
                        consumer_consent_provided TEXT,
                        submitted_via TEXT,
                        date_sent_to_company TEXT,
                        company_response_to_consumer TEXT,
                        timely_response TEXT,
                        consumer_disputed TEXT,
                        complaint_id INTEGER PRIMARY KEY
                        )''')
    cursor.execute(sql_query)

In [5]:
def drop_table(cursor):
    sql_query = sql.SQL(f'DROP TABLE IF EXISTS {TABLE_NAME};')
    cursor.execute(sql_query)

In [6]:
def get_products_list_by_company(cursor, company):
    sql_query = sql.SQL(f'''select * from {TABLE_NAME}
                        where company = '{company}' and state_name = (
                            select state_name from (
                                select state_name, count(*) amount_of_complaints from complaints c 
                                where company = '{company}' and state_name is not null
                                group by state_name 
                                order by amount_of_complaints desc limit 1
                            ) state_with_max_amount_of_issues
                        );''')
    cursor.execute(sql_query)
    return cursor.fetchall()

In [7]:
def get_products_list_by_date(cursor, start_date, end_date):
    sql_query = sql.SQL(f'''SELECT product_name, COUNT(issue) as amount_of_issues,
                            COUNT(CASE WHEN timely_response = 'Yes' THEN 1 END) as amount_of_timely_responded,
                            COUNT(CASE WHEN consumer_disputed = 'Yes' THEN 1 END) as amount_of_disputed
                            FROM {TABLE_NAME}
                            WHERE date_received >= '{start_date}'
                            AND date_received <= '{end_date}'
                            GROUP BY product_name
                            ORDER BY amount_of_issues''')
    cursor.execute(sql_query)
    return cursor.fetchall()

## Execution time comparison

### Load csv to postgres table vs to pandas DataFrame

In [8]:
%%timeit drop_table(cursor)

create_table(cursor)

with open('P9-ConsumerComplaints.csv', 'r') as f:
    next(f)
    cursor.copy_expert(f"""COPY {TABLE_NAME} FROM STDIN WITH (FORMAT CSV)""", f)

1.06 s ± 75.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit

df = pd.read_csv('P9-ConsumerComplaints.csv')

/home/amargaev/tools/anaconda3/lib/python3.8/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)
/home/amargaev/tools/anaconda3/lib/python3.8/site-packages/IPython/core/magics/execution.py:1167: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.
  all_runs = timer.repeat(repeat, number)


468 ms ± 9.83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Querying dataset postgres vs pandas

In [10]:
%%timeit company = 'Wells Fargo & Company'
res = get_products_list_by_company(cursor, company)

53.5 ms ± 537 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%%timeit company = 'Wells Fargo & Company'; df = pd.read_csv('P9-ConsumerComplaints.csv')
df[df['Company'] == company]

13 ms ± 123 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
%%timeit company = 'Wells Fargo & Company'
sql_query = sql.SQL(f'''select * from {TABLE_NAME}
                     where company = '{company}' and state_name = (
                         select state_name from (
                             select state_name, count(*) amount_of_complaints from complaints c 
                                 where company = '{company}' and state_name is not null
                                 group by state_name 
                                 order by amount_of_complaints desc limit 1
                             ) state_with_max_amount_of_issues
                         );''')
df = pd.read_sql(sql_query, conn)

74 ms ± 16.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit start_date = '2013-07-01'; end_date = '2013-09-01'
res = get_products_list_by_date(cursor, start_date, end_date)

18.9 ms ± 651 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%%timeit start_date = '2013-07-01'; end_date = '2013-09-01'; df = pd.read_csv('P9-ConsumerComplaints.csv')
df[(df['Date Received'] > start_date) & (df['Date Received'] < end_date)]

18.7 ms ± 72 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%%timeit start_date = '2013-07-01'; end_date = '2013-09-01'
sql_query = sql.SQL(f'''SELECT product_name, COUNT(issue) as amount_of_issues,
                        COUNT(CASE WHEN timely_response = 'Yes' THEN 1 END) as amount_of_timely_responded,
                        COUNT(CASE WHEN consumer_disputed = 'Yes' THEN 1 END) as amount_of_disputed
                        FROM {TABLE_NAME}
                        WHERE date_received >= '{start_date}'
                        AND date_received <= '{end_date}'
                        GROUP BY product_name
                        ORDER BY amount_of_issues''')
df = pd.read_sql(sql_query, conn)

19.2 ms ± 103 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
